In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
feature_data = SETTING["feature_data"]

if feature_data is None:

    feature_data = {}

else:

    for data_name, data_dict in feature_data.items():

        feature_data[data_name]["df"] = pd.read_table(
            data_dict["file_path"], index_col=0
        )

    print(feature_data.keys())

In [ ]:
sample_data = SETTING["sample_data"]

if sample_data is None:

    sample_data = {}

else:

    for data_name, data_dict in sample_data.items():

        sample_data[data_name]["df"] = pd.read_table(
            data_dict["file_path"], index_col=0
        )

    print(sample_data.keys())

In [ ]:
make_match_panels_kwargs = dict(
    n_job=SETTING["max_n_job"],
    n_required_for_match_function=SETTING["n_required_for_match_function"],
    extreme_feature_threshold=SETTING["extreme_feature_threshold"],
    n_sampling=SETTING["n_sampling"],
    n_permutation=SETTING["n_permutation"],
    plot_std=SETTING["plot_std"],
)

In [ ]:
for w_or_h, targets, data_dicts in (
    ("w", pd.read_table(PATH["w.tsv"], index_col=0).T, feature_data),
    ("h", pd.read_table(PATH["h.tsv"], index_col=0), sample_data),
):

    ccal.make_match_panels(
        targets,
        data_dicts,
        directory_path=PATH["{}|match/".format(w_or_h)],
        plotly_directory_path=PATH["plotly|{}_match/".format(w_or_h)],
        **make_match_panels_kwargs,
    )

In [ ]:
for w_or_h, data_dicts in (("w", feature_data), ("h", sample_data)):

    targets = pd.read_table(PATH["{}|cluster_x_column.tsv".format(w_or_h)], index_col=0)

    ccal.make_match_panels(
        targets,
        data_dicts,
        target_type="binary",
        cluster_within_category=targets.shape[1] < SETTING["plot_cluster_max_size"],
        directory_path=PATH["{}|hcc|match/".format(w_or_h)],
        plotly_directory_path=PATH["plotly|{}_match/".format(w_or_h)],
        **make_match_panels_kwargs,
    )